In [ ]:
import duckdb, os
from pathlib import Path

# Point to your unzipped PL state folder (contains geo + data CSVs)
DATA_DIR = Path("/path/to/your/unzipped/state_folder")  # e.g., Path("tx_pl")

# Where to save outputs
OUT_DIR = Path("data/processed"); OUT_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect((OUT_DIR / "pl_state.duckdb").as_posix())
print("DuckDB:", duckdb.__version__)


In [ ]:
# Heuristics to find GEO vs DATA files
geo_candidates  = sorted([p for p in DATA_DIR.glob("**/*.csv") if "geo" in p.name.lower()])
data_candidates = sorted([p for p in DATA_DIR.glob("**/*.csv") if "geo" not in p.name.lower()])

assert geo_candidates, "Couldn't find a GEO header CSV (name usually contains 'geo')."
assert data_candidates, "Couldn't find any data CSVs."

GEO_FILE  = geo_candidates[0]        # typically only one geo file
DATA_FILES = [p.as_posix() for p in data_candidates]

print("GEO:", GEO_FILE.name)
print("DATA files:", len(DATA_FILES))


In [ ]:
# Read GEO header: keep everything as VARCHAR first (safer)
con.execute("""
CREATE OR REPLACE TABLE st_geo AS
SELECT * FROM read_csv_auto(?, ALL_VARCHAR=TRUE);
""", [GEO_FILE.as_posix()])

# Read ALL data CSVs into one wide table
con.execute("""
CREATE OR REPLACE TABLE st_data AS
SELECT * FROM read_csv_auto(?, ALL_VARCHAR=TRUE)
""", [DATA_FILES])


In [ ]:
geo_cols  = {r[0] for r in con.execute("PRAGMA table_info('st_geo')").fetchall()}
data_cols = {r[0] for r in con.execute("PRAGMA table_info('st_data')").fetchall()}

# GEO essentials
must_geo = {"SUMLEV","LOGRECNO"}
assert must_geo.issubset(geo_cols), f"GEO file missing required columns: {must_geo - geo_cols}"

# Determine variable naming scheme
modern_needed = {"P1_001N","P3_001N","P4_003N"}
legacy_needed = {"P0010001","P0030001","P0040003"}

if modern_needed.issubset(data_cols):
    scheme = "modern"
elif legacy_needed.issubset(data_cols):
    scheme = "legacy"
else:
    raise RuntimeError("Couldn't detect PL column naming scheme (modern vs legacy). Inspect st_data columns.")
print("Detected scheme:", scheme)


In [ ]:
# Map variables for both schemes
if scheme == "modern":
    P1_TOTAL      = "P1_001N"
    P3_VAP        = "P3_001N"
    RACE_PARTS    = ["P3_003N","P3_004N","P3_005N","P3_006N","P3_007N","P3_008N","P3_009N"]  # White, Black, AIAN, Asian, NHPI, Other, 2+
    HISP_PARTS    = ["P4_002N","P4_003N"]  # Hispanic VAP, Not Hispanic VAP
else:
    # Legacy naming (P0010001 etc). These are the standard PL 2020 legacy codes.
    P1_TOTAL      = "P0010001"
    P3_VAP        = "P0030001"
    RACE_PARTS    = ["P0030003","P0030004","P0030005","P0030006","P0030007","P0030008","P0030009"]
    HISP_PARTS    = ["P0040002","P0040003"]

# Create a narrow data view with only the columns we need (+ LOGRECNO)
need_cols = [P1_TOTAL, P3_VAP] + RACE_PARTS + HISP_PARTS + ["LOGRECNO"]
sel_list  = ", ".join(need_cols)
con.execute(f"""
CREATE OR REPLACE TABLE st_data_narrow AS
SELECT {sel_list}
FROM st_data
""")

# Filter GEO to blocks (SUMLEV = '101'); keep useful geo fields if available
# GEO files vary by state/package; try to grab standard IDs when present.
geo_select = """
  g.LOGRECNO,
  g.SUMLEV,
  COALESCE(g.GEOID, g.GEOID20, g.BLKIDFP, g.BLOCKID) AS GEOID_any,
  COALESCE(g.STATE, g.STATEFP, g.STATEFP20)   AS statefp,
  COALESCE(g.COUNTY, g.COUNTYFP, g.COUNTYFP20) AS countyfp,
  COALESCE(g.TRACT, g.TRACTCE, g.TRACTCE20)  AS tractce,
  COALESCE(g.BLOCK, g.BLOCKCE, g.BLOCKCE20)  AS blockce
"""

con.execute(f"""
CREATE OR REPLACE TABLE st_geo_blocks AS
SELECT {geo_select}
FROM st_geo g
WHERE g.SUMLEV = '101'
""")

# Join on LOGRECNO to get block-level records with PL variables
con.execute("""
CREATE OR REPLACE TABLE pl_blocks_raw AS
SELECT
  gb.LOGRECNO,
  gb.GEOID_any,
  gb.statefp, gb.countyfp, gb.tractce, gb.blockce,
  d.*
FROM st_geo_blocks gb
JOIN st_data_narrow d USING (LOGRECNO)
""")


In [ ]:
# Build cast expressions
def cast_list(cols, as_type):
    return ",\n       ".join([f"CAST({c} AS {as_type}) AS {c}" for c in cols])

race_casts = cast_list(RACE_PARTS, "BIGINT")
hisp_casts = cast_list(HISP_PARTS, "BIGINT")

share_exprs = ",\n       ".join([
    f"CAST({c} AS DOUBLE)/NULLIF(CAST({P3_VAP} AS DOUBLE),0) AS {c}_share_vap"
    for c in (RACE_PARTS + HISP_PARTS)
])

con.execute("DROP TABLE IF EXISTS census_demographics_clean;")
con.execute(f"""
CREATE TABLE census_demographics_clean AS
WITH base AS (
  SELECT
    -- standardized GEOID20 (pick GEOID if present, otherwise build from parts)
    COALESCE(
      NULLIF(TRIM(GEOID_any), ''),
      statefp || countyfp || tractce || blockce
    )                                        AS GEOID20,
    statefp, countyfp, tractce, blockce,
    -- counts (casted)
    CAST({P1_TOTAL} AS BIGINT) AS P1_001N,
    CAST({P3_VAP}   AS BIGINT) AS P3_001N,
    {race_casts},
    {hisp_casts}
  FROM pl_blocks_raw
),
shares AS (
  SELECT
    *,
    {share_exprs}
  FROM base
)
SELECT * FROM shares;
""")

print(con.execute("SELECT COUNT(*) rows, COUNT(DISTINCT GEOID20) uniq FROM census_demographics_clean;").fetchdf())


In [ ]:
# Unique GEOID20
print(con.execute("""
SELECT COUNT(*) rows, COUNT(DISTINCT GEOID20) uniq
FROM census_demographics_clean;
""").fetchdf())

# GEOID20 looks like 15 digits
print(con.execute("""
SELECT SUM(LENGTH(GEOID20)=15 AND GEOID20 ~ '^[0-9]+$') AS ok_15,
       COUNT(*) AS rows
FROM census_demographics_clean;
""").fetchdf())

# Non-negative counts; VAP <= total pop
print(con.execute(f"""
SELECT
  SUM(({P1_TOTAL.replace('P0010001','P1_001N').replace('P1_001N','P1_001N')} < 0) OR (P3_001N < 0)) AS neg_pop,
  SUM(({ ' OR '.join([c+' < 0' for c in RACE_PARTS]) })) AS neg_race,
  SUM(({ ' OR '.join([c+' < 0' for c in HISP_PARTS]) }))  AS neg_hisp,
  AVG( (P3_001N <= P1_001N)::DOUBLE ) AS pct_vap_le_total
FROM census_demographics_clean;
""").fetchdf())


In [ ]:
parquet_path = OUT_DIR / "census_demographics.parquet"
csv_path     = OUT_DIR / "census_demographics.csv.gz"

con.execute(f"COPY census_demographics_clean TO '{parquet_path.as_posix()}' (FORMAT PARQUET);")
con.execute(f"COPY census_demographics_clean TO '{csv_path.as_posix()}' WITH (HEADER, DELIM ',', COMPRESSION 'gzip');")
print("Saved:", parquet_path, "and", csv_path)
